# Do interface design to further filter out unproductive designs prior to looping

In [1]:
# %load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers
running on node: dig62


In [2]:
os.chdir("/home/broerman/projects/crispy_shifty")

In [3]:
def create_tasks(selected, options):
    with open(selected, "r") as f:
        for line in f:
            tasks = {"options": "-corrections::beta_nov16 true"}
            tasks["extra_options"] = options
            tasks["pdb_path"] = line.rstrip()
            yield tasks

In [6]:

# Python standard library
import os
import pwd
import socket
import sys

# 3rd party library imports
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# Rosetta library imports
from pyrosetta.distributed.cluster.core import PyRosettaCluster

# Custom library imports
sys.path.insert(0, os.getcwd())
from crispy_shifty.protocols.design import one_state_design_unlooped_dimer  # the functions we will distribute

print(f"View dashboard at http://{socket.gethostname()}:8787")

selected = os.path.join(os.getcwd(), "projects/crispy_shifty_dimers/01_make_states/states.list")

options = {
    "-out:level": "100",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
    "-indexed_structure_store:fragment_store": "/net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
    "-precompute_ig": "true"
}

output_path = os.path.join(os.getcwd(), "projects/crispy_shifty_dimers/02_design_filter")
os.makedirs(output_path, exist_ok=True)

if __name__ == "__main__":
    # configure SLURM cluster as a context manager
    with SLURMCluster(
        cores=1,
        processes=1,
        job_cpu=1,
        memory="12GB",
        queue="medium",
        walltime="23:30:00",
        death_timeout=120,
        local_directory="$TMPDIR",  # spill worker litter on local node temp storage
        log_directory=os.path.join(output_path, "slurm_logs"),
        extra=["--lifetime", "23h", "--lifetime-stagger", "5m"],
    ) as cluster:
        print(cluster.job_script())
        # scale between 1-300 workers,
        cluster.adapt(
            minimum=1,
            maximum=2,
            wait_count=999,  # Number of consecutive times that a worker should be suggested for removal it is removed
            interval="5s",  # Time between checks
            target_duration="60s",
        )
        # setup a client to interact with the cluster as a context manager
        with Client(cluster) as client:
            print(client)
            client.upload_file(
                os.path.join(os.getcwd(), "crispy_shifty/protocols/design.py")
            )  # upload the script that contains the functions to distribute
            PyRosettaCluster(
                client=client,
                logging_level="WARNING",
                output_path=output_path,
                project_name="crispy_shifty_dimers",
                scratch_dir=output_path,
                simulation_name="02_design_filter",
                tasks=create_tasks(selected, options),
                nstruct=5,
            ).distribute(protocols=[one_state_design_unlooped_dimer])
            client.close()
        cluster.scale(0)
        cluster.close()
    print("distributed run complete")

View dashboard at http://dig62:8787
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /mnt/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter/slurm_logs/dask-worker-%J.err
#SBATCH -o /mnt/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter/slurm_logs/dask-worker-%J.out
#SBATCH -p medium
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=12G
#SBATCH -t 23:30:00

/projects/crispy_shifty/envs/crispy/bin/python -m distributed.cli.dask_worker tcp://172.16.131.92:34317 --nthreads 1 --memory-limit 11.18GiB --name dummy-name --nanny --death-timeout 120 --local-directory $TMPDIR --lifetime 23h --lifetime-stagger 5m --protocol tcp://

<Client: 'tcp://172.16.131.92:34317' processes=0 threads=0, memory=0 B>


KeyboardInterrupt: 

## Everything after this is for testing ##

In [2]:
# testing

sys.path.insert(0, "/home/broerman/projects/crispy_shifty")
from crispy_shifty.protocols.design import one_state_design_unlooped_dimer

pyrosetta.distributed.maybe_init(**{
    "options": "-corrections::beta_nov16 true",
    "extra_options":{
        "-out:level": "200",
        "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
        "-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
        "-precompute_ig": "true"
}})

import pyrosetta.distributed.io as io
file = '/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/01_make_states/decoys/0000/CSD_01_make_states_4c315eabb34a449a886b9e9bd9b8227a.pdb.bz2'

for i in tqdm(range(1)):
    pposes = one_state_design_unlooped_dimer(None, **{'pdb_path': file})
    for ppose in pposes:
        designed_pose = io.to_pose(ppose)
        # designed_pose.dump_pdb(f"/home/broerman/projects/dimeric_hinges/dimeric_hinges_2/testing/02_design_filter/X23_3_20_3_ct7_fc.pdb")
        designed_pose.dump_pdb(f"/home/broerman/projects/dimeric_hinges/dimeric_hinges_2/testing/02_design_filter/X23_3_20_3_ct7_fc_full-test_{i:02d}.pdb")

  0%|          | 0/1 [00:00<?, ?it/s]

0.00 min: Generated interface selector
0.00 min: Generated interface design task factory
0.06 min: Generated score functions


/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 95
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 288
select design_sel, (chain A and resid 2,6,28,31,32,35,38,39,42,46,49,50,51,52,54,55,56,58,59,61,62,65,66,69,73,75,77,78,80,81,82,84,85,87,88,89,90,91,92,93,94,95) or (chain B and resid 103,106,107,109,110,113,114,117,122,123,125,126,127,129,130,132,133,134,136,137,140,182) or (chain C and resid 245,247,248,251,252,254,255,256,258,259,262,263,265,266,268,270,271,274,275,277,278,279,281,282,284,285) or (chain D and resid 289,290,291,292,293,294,295,296,297,299,300,301,303,304,306,307,308,310,315,318,319,322,323,325,326,327,329,330,332,333,334,342,346,349,350,353,356,360)
0.22 min: Generating structure profile...complete.
4.26 min: Starting 1 round of fixed backbone design...core.chemical.AtomICoor: {0} [ WARNING ] IcoorAtomID::atom_id(): Cannot get atom_id for POLYMER_LOWER of